In [1]:
import json
import pandas as pd
import requests
from bs4  import BeautifulSoup

# obtian questions about python from page 1-500

In [2]:
def href(soup):
    # get all href links from one page 
    href=[]
    for i in soup.find_all("a",class_="question-hyperlink",href=True):
        href.append(i['href'])
    return href

In [3]:
def clean_empty_hrefs(hrefs):
   # remove all empty lists
    list_hrefs=[]
    for i in hrefs:
        if i!=[]:
            list_hrefs.append(i)
    # merge all elemenets in one list
    herfs_list=[]
    for i in list_hrefs:
        for j in i:
            herfs_list.append(j)
    return herfs_list

In [4]:
def add_prefix(herfs_list):
    # rearrage those links who do not have 'https://stackoverflow.com' prefix
    new_href=[]
    prefix='https://stackoverflow.com'
    for h in herfs_list:
        if 'https' not in h:
            m=prefix+h+"answertab=votes#tab-top"
            new_href.append(m)
        else:
            new_href.append(h+"answertab=votes#tab-top")
    return new_href

In [5]:
def single_page_scraper(url):
    req=requests.get(url=url)
    soup=BeautifulSoup(req.text,"html.parser")
    return soup

In [6]:
def single_page_question_answer(url):
    page=single_page_scraper(url).find_all("div",class_="post-text",itemprop="text")
    question=[i.find("p").get_text()for i in page][0]
    answer=[i.find("p").get_text() for i in page][1:3]
    
    return question,answer

In [7]:
import itertools

def questions_answers(start_page,end_page):
    soups=[]
    for page in range(start_page,end_page):
        req=requests.get(url='https://stackoverflow.com/questions/tagged/python?tab=votes&page={}&pagesize=15'.format(page))
        soup=BeautifulSoup(req.text,"html.parser")
        soups.append(soup)
    
    print("Soups are ready!")
    # obtain all href
    hrefs=[]
    for soup in soups:
        hrefs.append(href(soup))
    

    herfs_list=clean_empty_hrefs(hrefs)
    new_hrefs_list=add_prefix(herfs_list)
    print("All hrefs are ready!")

    quesitons=[]
    answers=[]
    for url in new_hrefs_list:
        try:
            q,a=single_page_question_answer(url)
            quesitons.append(q)
            answers.append(a)
        except:
            pass
    print("quesitons and answers are ready!")
    
    new_answers=[]
    for i in range(len(answers)):
        try:
            new_answers.append(answers[i][0])
        except:
            new_answers.append(None)
    print("All most done!")

    new_q = []
    new_a = []
    merge_answer=list(itertools.chain.from_iterable(answers))
    for i in range(len(merge_answer) - 1):
        new_q.append(merge_answer[i])
        new_a.append(merge_answer[i+1])
            
    return quesitons+new_q, new_answers+new_a

In [8]:
Questions,Answers=questions_answers(1,500)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [9]:
Questions1,Answers1=questions_answers(501,1000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [12]:
Questions2,Answers2=questions_answers(1000,2000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [14]:
Questions3,Answers3=questions_answers(2000,3000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [16]:
Questions4,Answers4=questions_answers(3000,4000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [27]:
Questions5,Answers5=questions_answers(4000,5000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [36]:
Questions6,Answers6=questions_answers(5000,6000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [43]:
Questions7,Answers7=questions_answers(6000,7000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [45]:
Questions8,Answers8=questions_answers(7000,8000)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [46]:
# add up all the data we collected above
As=Answers+Answers1+Answers2+Answers3+Answers4+Answers5+Answers6+Answers7+Answers8
Qs=Questions+Questions1+Questions2+Questions3+Questions4+Questions5+Questions6+Questions7+Questions8

In [47]:
# create a dataframe to store them togehter 
df=pd.DataFrame()
df["question"]=Qs
df["answer"]=As

In [48]:
df.head() # first five rows

,question,answer
0,If I try to use sudo kate to edit a system fil...,Just go ahead and edit the file without the su...
1,I'm writing a script in Bash that prompts the ...,"You may add -s to your read command, so it wou..."
2,If I try to use sudo kate to edit a system fil...,Just go ahead and edit the file without the su...
3,The Python requests module is simple and elega...,It is the underlying urllib3 library that does...
4,How can I check if a string has several specif...,Assuming your string is s:


In [49]:
# check the shape 
df.shape

(13557, 2)

In [50]:
# save as csv file
df.to_csv("df_python_stackoverflow.csv")